### GASS evolution

nb to test the evolution functions 

In [1]:
using PyCall
using DataFrames
using Printf

rootdir = "/home/stephane/Science/ALMA/ArrayConfig/GASS"
#rootdir = "/home/stephane/alma/ArrayConfig/GASS"

push!(LOAD_PATH,"$rootdir/master/src")
using GASS
using Printf

import PyPlot
import Random

@pyimport astropy.coordinates as coord
@pyimport astropy.modeling.models as am
@pyimport astropy.modeling.fitting as mf

@pyimport numpy as np


## directory
datadir = "$rootdir/master/data"
wdir    = "$rootdir/products"
plotdir = "$rootdir/products/test"

cd(wdir)

In [2]:
mutable struct _population
    age::Int32                 ## age
    subarr::Array{Array{Int,1},2}     ## {Population,Subindices}
    fitness::Array{Float64,2}  ## fitness of each subarray
    score::Array{Float64,1}    ## global score of each set
    param::Array{Dict{String,Float64},2}    ## beam and mrs for each subarray    
end

In [3]:
## fitness function for a subarray
## cfg: GASS parameters
## subarrid: subarray id (int)
## subind: subarray indices in the main Arr.
##
function _fitness_subarray(cfg, subarrid, subind)
    subarr = cfg.arr[subind,:]
    
    bl= calc_baselines(subarr)
    uv= calc_uv(bl, cfg.obs.Source_Hour_Angle ,  cfg.obs.Source_Declination)
    h , dr=  calc_dirtybeam(uv , 255, 127, robust=0.5)
    b= fit_beam(h , dr)
    mrs= calc_mrs(uv)
    
    #@printf("## subarray fitness \n")
    #@printf("## beam:")
    #println(b)
    #@printf("## MRS: %3.3f \n", mrs)
    
    res= 0
    res += cfg.wei.Weight_Spatial_Resolution[subarrid]*abs(b.ar-cfg.sub.Spatial_Resolution[subarrid])
    res += cfg.wei.Weight_Elongation[subarrid]*abs(b.e-cfg.sub.Elongation[subarrid])*sign(b.e-
        cfg.sub.Elongation[subarrid])
    res += cfg.wei.Weight_Sidelobe_Levels[subarrid]*abs(b.sidelobe-
        cfg.sub.Sidelobe_Level[subarrid])*sign(b.sidelobe-cfg.sub.Sidelobe_Level[subarrid])
    res += cfg.wei.Weight_Maximum_Recoverable_Scale[subarrid]*abs(mrs-
        cfg.sub.Maximum_Recoverable_Scale[subarrid])*sign(cfg.sub.Maximum_Recoverable_Scale[subarrid]-mrs)
    
    return(res)
end

_fitness_subarray (generic function with 1 method)

In [4]:
## Creation pf a population
## 

function _create_population(cfg)
    
    pop= Array{Array{Int,1},2}(undef,cfg.ga.Population_Size,  cfg.obs.Subarray_Number)
    fitness= Array{Float64,2}(undef,cfg.ga.Population_Size,  cfg.obs.Subarray_Number)
    score=  Array{Float64,1}(undef,cfg.ga.Population_Size)
    
    subind= collect(1:cfg.obs.Antenna_Number)
    
    for i in 1:cfg.ga.Population_Size
        Random.shuffle!(subind)
        for j in 1:cfg.obs.Subarray_Number
            pop[i,j]= subind[cfg.sub.Subrange[j]]
            fitness[i,j]= _fitness_subarray(cfg, j, pop[i,j])
            println(i," ",j," ",fitness[i,j])
        end
        score[i]= -sum(cfg.wei.Weight_Subarray[:] .* fitness[i,:])
    end
    
    _pop= _population(0, pop, fitness, score)
    
    return(_pop)
end

_create_population (generic function with 1 method)

In [5]:
## get best parents
##

function _get_elitism(cfg, pop::_population)
    isort= reverse(sortperm(pop.score))
    
    elit= []
    for i in 1:cfg.ga.Number_Elitism
        push!(elit, [pop.subarr[isort[i],:], pop.fitness[isort[i],:], pop.score[isort[i]]])
    end
    return(elit)
end

## using tournament selection
function _get_parents(cfg, pop::_population)
    tour= cfg.ga.Tournament_Size
    
    popran= Random.shuffle!(collect(1:cfg.ga.Population_Size))
    isort= reverse(sortperm(pop.score[popran[1:tour]]))
    iparent1= popran[isort[1]]
  
    iparent2= iparent1
    while iparent1==iparent2
        popran= Random.shuffle!(collect(1:cfg.ga.Population_Size))
        isort= reverse(sortperm(pop.score[popran[1:tour]]))
        iparent2= popran[isort[1]] 
    end 
    return(iparent1 , iparent2)
end

_get_parents (generic function with 1 method)

In [6]:
function _flatten_subarray(cfg, subarr)
    subflat= zeros(Int, cfg.obs.Antenna_Number)
    
    counter= 1
    for i in 1:cfg.obs.Subarray_Number
        for j in 1:cfg.sub.Pads_Per_Subarray[i]
            subflat[counter]= subarr[i][j]
            counter += 1
        end
    end
    return(subflat)
end

## reform the subarray array..
function _wrap_subarray(cfg, sub)
    subarray= Array{Array{Int,1}}(undef,0)

    for i in 1:cfg.obs.Subarray_Number
        push!(subarray, sub[cfg.sub.Subrange[i]])
    end
    
    return(subarray)
end

## crossover using the two parents
##
function _get_crossover1(cfg, pop::_population , parents)
    nmax= cfg.obs.Antenna_Number
    p1= _flatten_subarray(cfg, pop.subarr[parents[1],:])
    p2= _flatten_subarray(cfg, pop.subarr[parents[2],:])
    child= zeros(Int, nmax)

    pivot= rand(1:nmax,2)
    pivot1= minimum(pivot) ; pivot2= maximum(pivot)
    child[pivot1:pivot2] .= p1[pivot1:pivot2]

    irange1= 1:pivot1-1 ; irange2= pivot2+1:nmax
    
    cnterp2= 1
    for i in irange1
        while (p2[cnterp2] in child)
            cnterp2 += 1
        end
        child[i]= p2[cnterp2]
    end
    for i in irange2
        while (p2[cnterp2] in child)
            cnterp2 += 1
        end
        child[i]= p2[cnterp2]
    end  
    
    # println(child)
    c= _wrap_subarray(cfg, child)
    # println(c)
    return(child)
end

_get_crossover1 (generic function with 1 method)

In [7]:
## mutation
## loop over all alleles and swap two if p is true
## 

function _get_mutation(cfg, child)
    nmax= cfg.obs.Antenna_Number
    si= copy(child)
    
    pmutation= 1-cfg.ga.Mutation_Rate
    rng = Random.MersenneTwister()
    rd= Random.rand(rng, nmax)
    # println(rd)
    for i in 1:nmax
        if rd[i] > pmutation
            iswap= Random.rand(rng, big.(1:nmax))
            value= si[iswap]
            si[iswap]= si[i]
            si[i]= value
        end
    end
    return(si)
end

_get_mutation (generic function with 1 method)

In [8]:
## Evolve one population
##

function _get_evolution(cfg, pi::_population)
    npop= cfg.ga.Population_Size
    age= pi.age + 1
    nelit= cfg.ga.Number_Elitism
    
    ## Elitism
    pelit=  _get_elitism(cfg, pi)
    
    ## parent selection, crossover and mutation
    ncross= npop-nelit
    
    crosspop= []
    for i in 1:ncross
        parent= _get_parents(cfg, pi)
        child=  _get_crossover1(cfg, pi , parent)
        mutated= _get_mutation(cfg, child)
        push!(crosspop, mutated)
        println(parent)
        println(child)
    end

    
    ## create an evolved population
    pinew= Array{Array{Int,1},2}(undef,cfg.ga.Population_Size,  cfg.obs.Subarray_Number)
    fitness= Array{Float64,2}(undef,cfg.ga.Population_Size,  cfg.obs.Subarray_Number)
    score=  Array{Float64,1}(undef,cfg.ga.Population_Size)
    
    for i in 1:nelit
        for j in 1:cfg.obs.Subarray_Number
            pinew[i,j]= pelit[i][1][j]
            fitness[i,j]= _fitness_subarray(cfg, j, pinew[i,j])
            println(i," ",j," ",fitness[i,j])
        end
        score[i]= -sum(cfg.wei.Weight_Subarray[:] .* fitness[i,:])
    end    
    
    println("fitness of crossover..")
    for i in nelit+1:npop
        println(crosspop[i-nelit])
        pwrap= _wrap_subarray(cfg, crosspop[i-nelit])
        println(pwrap)
        for j in 1:cfg.obs.Subarray_Number
            pinew[i,j]= pwrap[j]
            fitness[i,j]= _fitness_subarray(cfg, j, pinew[i,j])
            println(i," ",j," ",fitness[i,j])
        end
        score[i]= -sum(cfg.wei.Weight_Subarray[:] .* fitness[i,:])
    end
    
    _pop= _population(age, pinew, fitness, score)
    
    return(_pop)
end

_get_evolution (generic function with 1 method)

In [9]:
macro main(inpfile)
    cfg = read_cfg(inpfile , verbose=true)
    
    println("##")
    println("## Creating the population...")
    p0= _create_population(cfg)
    
    println("Elitism..")
    pelit=  _get_elitism(cfg, p0)
    
    println("Crossover...")
    parent= _get_parents(cfg, p0)
    child=  _get_crossover1(cfg, p0 , parent)
    println(child)
    
    println("Mutatis mutandis...")
    mutated= _get_mutation(cfg, child)
    
    ## evolution
    println("Evolution...")
    
    species= []
    for i in 1:cfg.ga.Number_Iterations
        p1= _get_evolution(cfg, p0)
        push!(species,p1)
        p0=p1
        if (i % 10) == 0
            printf("## Iteration: %d",i)
        end
    end
    
    
    ## PLOT ####
    #PyPlot.imshow(h)
    #PyPlot.colorbar()
    #PyPlot.show()
    
end

@main("../master/data/GA_Inputs_O-3.txt.julia")

## Input Parameters for GASS 
### Configuration file: ../master/data/O-3.cfg 
### Obs. Latitude: -23.026 
### Source Declination: -50.0 
### HA: 0.0 
### Antenna number: 50 
### Subarray number: 4 
##
## Subarray Parameters
### Pads per subarray: [30, 10, 7, 3]
### Name: ["Sub_1", " Sub_2 ", " Sub_3 ", " Sub_4"]
### AR: [2.3, 3.0, 3.5, 3.5]
### MRS: [20.0, 20.0, 10.0, 10.0]
### elongation: [1.2, 1.3, 2.0, 3.0]
### sidelobe: [10.0, 20.0, 60.0, 60.0]
##
## GA parameters
### Iterations: 10 
### Population size: 100 
### Mutation rate: 0.050 
### Tournament size: 5 
### Elitism: 5 
##
## Weights
### Subarray weights: [0.25, 0.25, 0.25, 0.25]
### AR weights: [0.25, 0.25, 0.25, 0.25]
### MRS weights: [0.25, 0.25, 0.25, 0.25]
### elongation weights: [0.25, 0.25, 0.25, 0.25]
### sidelobe weights: [0.25, 0.25, 0.25, 0.25]
##
##
## Creating the population...
1 1 -0.5587785391508211
1 2 -0.767405945272968
1 3 -11.84252611047258
1 4 -9.671891676333374
2 1 -8.591460158283969
2 2 -4.0946079349126405

77 2 -0.5369768874944261
77 3 -10.520265996190938
77 4 2.8939195494969927
78 1 3.233184601912664
78 2 -0.9123492627285916
78 3 -12.556979626463441
78 4 16.1214461570451
79 1 -14.987583487697801
79 2 -0.8466061405061471
79 3 -12.978830132381265
79 4 4.158824495130528
80 1 2.73563981022345
80 2 -8.968981846036364
80 3 -14.123922869495537
80 4 -16.31467106241024
81 1 2.017508601946672
81 2 -0.7097860566314282
81 3 -13.743012109370767
81 4 -12.333701658579171
82 1 2.3774439934417724
82 2 1.3637856501911267
82 3 -11.729205732723806
82 4 6.758715209383574
83 1 1.8909810849458848
83 2 -5.729513538852197
83 3 -11.926990806075061
83 4 12.79855136869462
84 1 2.354905590011134
84 2 1.4595821499167894
84 3 -12.043933773815304
84 4 -10.95515711432011
85 1 1.680769763839272
85 2 -4.216642912680976
85 3 -10.87027437399944
85 4 4.4347046882852315
86 1 1.330239450766669
86 2 2.1789546861882965
86 3 -12.635825247344798
86 4 -13.107010231365486
87 1 -4.6195519733478925
87 2 -1.064264650706674
87 3 -12.58

(28, 24)
[3, 22, 41, 5, 43, 30, 28, 11, 47, 45, 2, 23, 25, 39, 44, 18, 20, 6, 38, 17, 7, 8, 1, 12, 21, 35, 34, 16, 48, 15, 29, 24, 13, 33, 37, 9, 19, 10, 50, 46, 32, 31, 36, 49, 40, 42, 26, 4, 14, 27]
(58, 13)
[10, 28, 34, 42, 32, 20, 11, 13, 23, 48, 36, 47, 50, 17, 19, 40, 16, 27, 14, 22, 41, 25, 8, 9, 33, 7, 24, 2, 49, 3, 29, 37, 45, 46, 18, 26, 31, 39, 43, 38, 1, 30, 6, 12, 21, 44, 4, 5, 35, 15]
(79, 76)
[2, 6, 9, 10, 30, 15, 33, 7, 44, 47, 22, 39, 18, 16, 5, 20, 14, 11, 40, 26, 50, 48, 43, 37, 21, 31, 32, 25, 12, 49, 45, 29, 13, 34, 42, 35, 38, 23, 3, 17, 46, 4, 8, 36, 28, 24, 27, 1, 41, 19]
(77, 11)
[43, 15, 46, 21, 1, 41, 26, 2, 44, 20, 13, 37, 16, 49, 42, 10, 34, 27, 47, 29, 18, 39, 7, 8, 11, 33, 50, 17, 31, 24, 6, 4, 12, 48, 14, 23, 30, 9, 19, 28, 45, 3, 25, 32, 22, 36, 40, 35, 38, 5]
(7, 81)
[21, 38, 48, 14, 44, 4, 41, 32, 49, 43, 19, 18, 12, 30, 37, 7, 5, 29, 45, 35, 27, 8, 42, 47, 2, 40, 10, 50, 17, 25, 22, 34, 36, 39, 33, 24, 3, 6, 28, 31, 20, 26, 9, 13, 11, 23, 16, 1, 46, 

13 1 2.160919516429038
13 2 -0.3031330219530344
13 3 -18.27912153517768
13 4 6.57659710616505
[12, 48, 2, 24, 22, 14, 45, 9, 47, 1, 34, 4, 21, 25, 6, 42, 19, 41, 43, 8, 18, 28, 16, 29, 27, 10, 7, 15, 44, 50, 46, 3, 37, 33, 11, 39, 38, 49, 13, 23, 32, 5, 40, 35, 26, 20, 36, 17, 31, 30]
Array{Int64,1}[[12, 48, 2, 24, 22, 14, 45, 9, 47, 1, 34, 4, 21, 25, 6, 42, 19, 41, 43, 8, 18, 28, 16, 29, 27, 10, 7, 15, 44, 50], [46, 3, 37, 33, 11, 39, 38, 49, 13, 23], [32, 5, 40, 35, 26, 20, 36], [17, 31, 30]]
14 1 -2.532644415379666
14 2 2.8307841010395887
14 3 -14.038376173282703
14 4 -13.032589237412145
[42, 12, 48, 44, 24, 22, 14, 45, 9, 47, 6, 1, 43, 18, 16, 17, 13, 50, 34, 25, 49, 29, 27, 10, 7, 15, 2, 21, 46, 19, 3, 37, 33, 11, 39, 23, 28, 38, 32, 5, 40, 35, 26, 20, 36, 41, 8, 31, 4, 30]
Array{Int64,1}[[42, 12, 48, 44, 24, 22, 14, 45, 9, 47, 6, 1, 43, 18, 16, 17, 13, 50, 34, 25, 49, 29, 27, 10, 7, 15, 2, 21, 46, 19], [3, 37, 33, 11, 39, 23, 28, 38, 32, 5], [40, 35, 26, 20, 36, 41, 8], [31, 4, 3

30 1 0.5925760268206273
30 2 -1.064264650706674
30 3 -12.984205325695605
30 4 -8.946215308091352
[20, 1, 27, 26, 46, 31, 45, 32, 40, 21, 25, 6, 42, 19, 15, 3, 13, 49, 28, 12, 48, 43, 4, 9, 41, 8, 17, 35, 44, 47, 50, 11, 10, 36, 7, 38, 39, 33, 2, 34, 24, 29, 22, 16, 30, 5, 23, 18, 14, 37]
Array{Int64,1}[[20, 1, 27, 26, 46, 31, 45, 32, 40, 21, 25, 6, 42, 19, 15, 3, 13, 49, 28, 12, 48, 43, 4, 9, 41, 8, 17, 35, 44, 47], [50, 11, 10, 36, 7, 38, 39, 33, 2, 34], [24, 29, 22, 16, 30, 5, 23], [18, 14, 37]]
31 1 -0.6282433157005611
31 2 1.3405589555740591
31 3 -12.076633331072408
31 4 -9.671891676333374
[6, 10, 21, 13, 3, 11, 7, 48, 33, 46, 45, 14, 50, 44, 28, 1, 35, 38, 34, 17, 36, 41, 23, 40, 31, 20, 32, 22, 2, 5, 42, 30, 27, 29, 9, 18, 47, 24, 39, 25, 4, 49, 26, 19, 37, 15, 16, 12, 8, 43]
Array{Int64,1}[[6, 10, 21, 13, 3, 11, 7, 48, 33, 46, 45, 14, 50, 44, 28, 1, 35, 38, 34, 17, 36, 41, 23, 40, 31, 20, 32, 22, 2, 5], [42, 30, 27, 29, 9, 18, 47, 24, 39, 25], [4, 49, 26, 19, 37, 15, 16], [12, 8

...

32 1 2.025649145161586
32 2 -1.8686171105643092
32 3 -12.550257822494082
32 4 -1.6011378660398852
[42, 4, 11, 6, 5, 26, 1, 43, 28, 9, 23, 48, 8, 14, 40, 16, 18, 10, 7, 15, 3, 45, 12, 35, 17, 41, 13, 50, 34, 25, 49, 27, 2, 21, 46, 32, 19, 38, 44, 33, 24, 47, 36, 37, 29, 39, 20, 31, 22, 30]
Array{Int64,1}[[42, 4, 11, 6, 5, 26, 1, 43, 28, 9, 23, 48, 8, 14, 40, 16, 18, 10, 7, 15, 3, 45, 12, 35, 17, 41, 13, 50, 34, 25], [49, 27, 2, 21, 46, 32, 19, 38, 44, 33], [24, 47, 36, 37, 29, 39, 20], [31, 22, 30]]
33 1 1.9802133361291272
33 2 -0.11548263395641811
33 3 -11.436271000472253
33 4 -635.3058035235825
[18, 15, 6, 23, 43, 48, 33, 29, 12, 27, 10, 16, 39, 34, 35, 38, 28, 44, 24, 7, 2, 40, 46, 19, 37, 32, 9, 13, 50, 4, 8, 14, 3, 20, 45, 47, 5, 42, 11, 1, 25, 49, 17, 26, 21, 36, 41, 31, 22, 30]
Array{Int64,1}[[18, 15, 6, 23, 43, 48, 33, 29, 12, 27, 10, 16, 39, 34, 35, 38, 28, 44, 24, 7, 2, 40, 46, 19, 37, 32, 9, 13, 50, 4], [8, 14, 3, 20, 45, 47, 5, 42, 11, 1], [25, 49, 17, 26, 21, 36, 41], [31, 

49 1 -1.548200304060373
49 2 0.6820241260342552
49 3 -12.017418519915934
49 4 -635.3058035235825
[3, 38, 6, 43, 32, 16, 36, 41, 50, 35, 34, 33, 28, 9, 26, 40, 11, 39, 12, 29, 49, 20, 21, 10, 7, 15, 44, 27, 18, 46, 2, 23, 19, 4, 17, 37, 13, 47, 14, 24, 45, 5, 42, 1, 25, 48, 8, 31, 22, 30]
Array{Int64,1}[[3, 38, 6, 43, 32, 16, 36, 41, 50, 35, 34, 33, 28, 9, 26, 40, 11, 39, 12, 29, 49, 20, 21, 10, 7, 15, 44, 27, 18, 46], [2, 23, 19, 4, 17, 37, 13, 47, 14, 24], [45, 5, 42, 1, 25, 48, 8], [31, 22, 30]]
50 1 0.7169802524677822
50 2 -1.6669086027089133
50 3 -21.893513988197014


Excessive output truncated after 524289 bytes.

50 4 -635.3058035235825
[37, 6, 43, 9, 25, 18, 20, 16, 17, 1, 50, 14, 45, 5, 34, 28, 38, 24, 33, 4,

LoadError: UndefVarError: printf not defined